<a href="https://colab.research.google.com/github/lunaB/AI-Study/blob/master/11_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
torch.manual_seed(0)

In [2]:
sentence = "but there is no shop anywhere where one can buy friendship, and so men have no friends any more."
print(len(sentence))
x_data = []
y_data = []
seq_len = 10

char_set = list(set(sentence))
char_dict = {c:i for i, c in enumerate(char_set)}

print(char_dict)

for i in range(len(sentence)-seq_len):
    x_str = sentence[i:i+seq_len]
    y_str = sentence[i+1:i+seq_len+1]
    x_data.append([char_dict[c] for c in x_str])
    y_data.append([char_dict[c] for c in y_str])

one_hot = [np.eye(len(char_dict))[i] for i in x_data]

X = torch.FloatTensor(one_hot)
Y = torch.LongTensor(y_data)

96
{'e': 0, 'r': 1, 't': 2, 'v': 3, 'y': 4, 'm': 5, '.': 6, 'n': 7, 'a': 8, 'b': 9, ',': 10, 'p': 11, 's': 12, 'o': 13, 'c': 14, 'f': 15, 'i': 16, 'w': 17, 'u': 18, 'd': 19, ' ': 20, 'h': 21}


In [3]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(len(char_dict), len(char_dict), 2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

for i in range(1000):
    optimizer.zero_grad()
    outputs = net(X)
    # print(outputs.shape)
    # print(outputs.view(-1, len(sent_dict)).shape)
    # print(Y.shape)
    loss = criterion(outputs.view(-1, len(char_dict)), Y.view(-1))
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
        pred_str = sentence[0]
        pred_vec = outputs.data.numpy().argmax(axis=2)
        for i, v in enumerate(pred_vec):
            if i == 0:
                pred_str += ''.join([char_set[j] for j in v])
            else:
                pred_str += char_set[v[-1]]
        print(loss.item(), pred_str, pred_str==sentence)

0.260539710521698 buy there is no shop anywhere where one can buy friendship, and so men have no friendshany morer False
0.18756131827831268 buy there is no shop anywhere where one can buy friendship, and so men have no friends any more. False
0.18016597628593445 buy there is no shop anywhere where one can buy friendship, and so men have no friends any more. False
0.1778877228498459 buy there is no shop anywhere where one can buy friendship, and so men have no friends any more. False
0.17650577425956726 buy there is no shop anywhere where one can buy friendship, and so men have no friends any more. False
0.17576725780963898 buy there is no shop anywhere where one can buy friendship, and so men have no friends any more. False
0.17530053853988647 but there is no shop anywhere where one can buy friendship, and so men have no friends any more. True
0.1750594675540924 but there is no shop anywhere where one can buy friendship, and so men have no friends any more. True
0.17480942606925964 bu